In [21]:
import os
import numpy as np
import pandas as pd

In [22]:
def read_excel(path):
    return pd.read_excel(path)

def delete_outlier(df, HARI, JAM):
    df = df[(df['HARI'] == HARI) & (df['JAM'] == JAM)]    
    
    # Menghitung Q1 (kuartil pertama) dan Q3 (kuartil ketiga)
    Q1 = df['SMP'].quantile(0.25)
    Q3 = df['SMP'].quantile(0.75)
    IQR = Q3 - Q1
    
    # Menentukan batas bawah dan batas atas untuk outlier
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Menemukan dan menghapus outlier
    df_outlier = df[(df['SMP'] < lower_bound) | (df['SMP'] > upper_bound)]
    df_cleaned = df[(df['SMP'] >= lower_bound) & (df['SMP'] <= upper_bound)]
    
    # if(len(df_outlier)>0):
    #     print('Ada Outlier')

    # df_grouped = df_cleaned[['HARI', 'JAM', 'SMP']]
    # df_grouped = round(df_grouped.groupby(['HARI', 'JAM']).mean()).reset_index()

    return df_outlier, df_cleaned


In [23]:
path_1 = '../Data/Preprocessing/Data Losarang/2023/2_input_losarang_2023.xlsx'

df = read_excel(path_1)
df.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 00:00:00,2023,1,1,Sunday,00:00,85.0,24.0,4.0,113.0,17.0,24.0,10.0,51.0
1,2023-01-01 00:15:00,2023,1,1,Sunday,00:15,92.0,19.0,6.0,117.0,18.4,19.0,15.0,52.0
2,2023-01-01 00:30:00,2023,1,1,Sunday,00:30,78.0,16.0,6.0,100.0,15.6,16.0,15.0,47.0
3,2023-01-01 00:45:00,2023,1,1,Sunday,00:45,78.0,23.0,4.0,105.0,15.6,23.0,10.0,49.0
4,2023-01-01 01:00:00,2023,1,1,Sunday,01:00,76.0,14.0,6.0,96.0,15.2,14.0,15.0,44.0


In [24]:
df_waktu = df[['HARI','JAM']]
df_waktu = df_waktu.groupby(['HARI', 'JAM']).mean().round().reset_index()
df_waktu.head()

,HARI,JAM
0,Friday,00:00
1,Friday,00:15
2,Friday,00:30
3,Friday,00:45
4,Friday,01:00


In [25]:
all_df_outlier = []
all_df_cleaned = []

for i, row in df_waktu.iterrows():
    df_outlier,  df_cleaned = delete_outlier(df, row['HARI'], row['JAM'])
    all_df_outlier.append(df_outlier)
    all_df_cleaned.append(df_cleaned)

# gabungkan data
df_outlier_combined = pd.concat(all_df_outlier)
df_cleaned_combined = pd.concat(all_df_cleaned)

In [26]:
df_outlier_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
480,2023-01-06,2023,1,6,Friday,00:00,28.0,45.0,43.0,116.0,5.6,45.0,107.5,158.0
1152,2023-01-13,2023,1,13,Friday,00:00,29.0,47.0,35.0,111.0,5.8,47.0,87.5,140.0
1824,2023-01-20,2023,1,20,Friday,00:00,23.0,44.0,39.0,106.0,4.6,44.0,97.5,146.0
8544,2023-03-31,2023,3,31,Friday,00:00,21.0,30.0,65.0,116.0,4.2,30.0,162.5,197.0
10560,2023-04-21,2023,4,21,Friday,00:00,1276.0,151.0,23.0,1450.0,255.2,151.0,57.5,464.0


In [27]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
2496,2023-01-27,2023,1,27,Friday,00:00,104.0,75.0,102.0,281.0,20.8,75.0,255.0,351.0
3168,2023-02-03,2023,2,3,Friday,00:00,104.0,75.0,102.0,281.0,20.8,75.0,255.0,351.0
3840,2023-02-10,2023,2,10,Friday,00:00,104.0,75.0,102.0,281.0,20.8,75.0,255.0,351.0
4512,2023-02-17,2023,2,17,Friday,00:00,104.0,75.0,102.0,281.0,20.8,75.0,255.0,351.0
5184,2023-02-24,2023,2,24,Friday,00:00,104.0,75.0,102.0,281.0,20.8,75.0,255.0,351.0


In [28]:
df_outlier_combined = df_outlier_combined.sort_values(by='WAKTU').reset_index(drop=True)
df_cleaned_combined = df_cleaned_combined.sort_values(by='WAKTU').reset_index(drop=True)

In [29]:
df_cleaned_combined.to_excel('../Data/Preprocessing/Data Losarang/2023/3_clean_losarang_2023.xlsx', index=False)
df_outlier_combined.to_excel('../Data/Preprocessing/Data Losarang/2023/3_outliers_losarang_2023.xlsx', index=False)

In [30]:
df_cleaned_combined.head()

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,JUMLAH,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 08:45:00,2023,1,1,Sunday,08:45,216.0,82.0,44.0,342.0,43.2,82.0,110.0,235.0
1,2023-01-01 09:15:00,2023,1,1,Sunday,09:15,192.0,113.0,38.0,343.0,38.4,113.0,95.0,246.0
2,2023-01-01 09:30:00,2023,1,1,Sunday,09:30,133.0,98.0,42.0,273.0,26.6,98.0,105.0,230.0
3,2023-01-01 10:45:00,2023,1,1,Sunday,10:45,202.0,108.0,53.0,363.0,40.4,108.0,132.5,281.0
4,2023-01-01 11:00:00,2023,1,1,Sunday,11:00,236.0,107.0,35.0,378.0,47.2,107.0,87.5,242.0


In [31]:
df_smp = df_cleaned_combined[['WAKTU','TAHUN', 'BULAN', 'TANGGAL', 'HARI','JAM','MOTOR','MOBIL','TRUK/BUS',
                              'SMP_MTR','SMP_MBL','SMP_TRK','SMP']]

In [32]:
df_smp.head(8)

,WAKTU,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023-01-01 08:45:00,2023,1,1,Sunday,08:45,216.0,82.0,44.0,43.2,82.0,110.0,235.0
1,2023-01-01 09:15:00,2023,1,1,Sunday,09:15,192.0,113.0,38.0,38.4,113.0,95.0,246.0
2,2023-01-01 09:30:00,2023,1,1,Sunday,09:30,133.0,98.0,42.0,26.6,98.0,105.0,230.0
3,2023-01-01 10:45:00,2023,1,1,Sunday,10:45,202.0,108.0,53.0,40.4,108.0,132.5,281.0
4,2023-01-01 11:00:00,2023,1,1,Sunday,11:00,236.0,107.0,35.0,47.2,107.0,87.5,242.0
5,2023-01-01 12:00:00,2023,1,1,Sunday,12:00,191.0,132.0,39.0,38.2,132.0,97.5,268.0
6,2023-01-01 14:00:00,2023,1,1,Sunday,14:00,204.0,75.0,35.0,40.8,75.0,87.5,203.0
7,2023-01-01 16:00:00,2023,1,1,Sunday,16:00,227.0,65.0,24.0,45.4,65.0,60.0,170.0


In [33]:
# Round down the WAKTU column to the nearest hour
df_smp_jam = df_smp.copy()
df_smp_jam['JAM'] = df_smp['WAKTU'].dt.floor('H')

# Drop the original WAKTU column if it's not needed anymore
df_smp_jam = df_smp_jam.drop(columns=['WAKTU'])

# Format the 'JAM' column to show only hour and minute
df_smp_jam['JAM'] = df_smp_jam['JAM'].dt.strftime('%H:%M')

# Group by the rounded-down hour and sum the SMP column
df_smp_jam = df_smp_jam.groupby(['TAHUN', 'BULAN', 'TANGGAL', 'HARI', 'JAM'], as_index=False).mean().round()

In [34]:
df_smp_jam.head(8)

,TAHUN,BULAN,TANGGAL,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,2023,1,1,Sunday,08:00,216.0,82.0,44.0,43.0,82.0,110.0,235.0
1,2023,1,1,Sunday,09:00,162.0,106.0,40.0,33.0,106.0,100.0,238.0
2,2023,1,1,Sunday,10:00,202.0,108.0,53.0,40.0,108.0,132.0,281.0
3,2023,1,1,Sunday,11:00,236.0,107.0,35.0,47.0,107.0,88.0,242.0
4,2023,1,1,Sunday,12:00,191.0,132.0,39.0,38.0,132.0,98.0,268.0
5,2023,1,1,Sunday,14:00,204.0,75.0,35.0,41.0,75.0,88.0,203.0
6,2023,1,1,Sunday,16:00,225.0,79.0,30.0,45.0,79.0,75.0,199.0
7,2023,1,2,Monday,06:00,101.0,53.0,35.0,20.0,53.0,88.0,161.0


In [35]:
df_lhrt = df_smp_jam.copy()

In [36]:
df_lhrt = df_lhrt.drop(columns=['TAHUN','BULAN','TANGGAL'])

In [37]:
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Sunday,08:00,216.0,82.0,44.0,43.0,82.0,110.0,235.0
1,Sunday,09:00,162.0,106.0,40.0,33.0,106.0,100.0,238.0
2,Sunday,10:00,202.0,108.0,53.0,40.0,108.0,132.0,281.0
3,Sunday,11:00,236.0,107.0,35.0,47.0,107.0,88.0,242.0
4,Sunday,12:00,191.0,132.0,39.0,38.0,132.0,98.0,268.0


In [38]:
df_lhrt  = df_lhrt.groupby(['HARI', 'JAM'], as_index=False).mean().round()
df_lhrt.head()

,HARI,JAM,MOTOR,MOBIL,TRUK/BUS,SMP_MTR,SMP_MBL,SMP_TRK,SMP
0,Friday,00:00,78.0,69.0,103.0,16.0,69.0,258.0,342.0
1,Friday,01:00,92.0,64.0,93.0,18.0,64.0,231.0,315.0
2,Friday,02:00,57.0,56.0,79.0,11.0,56.0,197.0,264.0
3,Friday,03:00,42.0,54.0,74.0,9.0,54.0,184.0,247.0
4,Friday,04:00,41.0,47.0,68.0,8.0,47.0,170.0,226.0


In [39]:
len(df_lhrt)

168

In [40]:
df_lhrt.to_excel('../Data/Preprocessing/Data Losarang/2023/3_lhrt_losarang_2023.xlsx', index=False)